In [187]:
#Imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
from collections import Counter
import requests
import logging

logging.getLogger("transformers").setLevel(logging.ERROR)

print("complete")

complete


In [188]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return text.split()

def build_vocab(word_list, vocab_size=5000):
    most_common = Counter(word_list).most_common(vocab_size - 1)
    vocab = {w: i+1 for i, (w, _) in enumerate(most_common)}
    vocab["<UNK>"] = 0
    return vocab

def encode_words(word_list, vocab):
    return [vocab.get(word, vocab["<UNK>"]) for word in word_list]

url = "https://www.gutenberg.org/files/11/11-0.txt"
response = requests.get(url)
raw_text = response.text

words = preprocess_text(raw_text)
vocab = build_vocab(words)               
encoded = encode_words(words, vocab)
idx2word = {i: w for w, i in vocab.items()}  

In [189]:

class SimpleGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SimpleGRU, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.W_z = nn.Linear(input_dim, hidden_dim)
        self.U_z = nn.Linear(hidden_dim, hidden_dim, bias=False)

        self.W_r = nn.Linear(input_dim, hidden_dim)
        self.U_r = nn.Linear(hidden_dim, hidden_dim, bias=False)

        self.W_h = nn.Linear(input_dim, hidden_dim)
        self.U_h = nn.Linear(hidden_dim, hidden_dim, bias=False)

    def forward(self, x, h_0=None):
        seq_len, batch_size, _ = x.size()
        if h_0 is None:
            h_t = torch.zeros(batch_size, self.hidden_dim, device=x.device)
        else:
            h_t = h_0

        outputs = []
        for t in range(seq_len):
            x_t = x[t]
            z_t = torch.sigmoid(self.W_z(x_t) + self.U_z(h_t))
            r_t = torch.sigmoid(self.W_r(x_t) + self.U_r(h_t))
            h_tilde = torch.tanh(self.W_h(x_t) + self.U_h(r_t * h_t))
            h_t = (1 - z_t) * h_t + z_t * h_tilde
            outputs.append(h_t.unsqueeze(0))
        return torch.cat(outputs, dim=0), h_t


class GRULanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRULanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = SimpleGRU(embedding_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        gru_out, _ = self.gru(embedded)
        logits = self.decoder(gru_out[-1])
        return logits


In [190]:
class NeuralLM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1]) 
        return out

vocab_size = len(vocab)
model = NeuralLM(vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()
print("complete")

complete


In [191]:

from torch.utils.data import Dataset, DataLoader

class LanguageModelDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx : idx + self.seq_len])
        y = torch.tensor(self.data[idx + self.seq_len])
        return x, y


dataset = LanguageModelDataset(encoded, seq_len=5)
loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

In [192]:
for epoch in range(10):
    total_loss = 0
    for inputs, targets in loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")
    torch.save(model.state_dict(), f"neural_lm_epoch{epoch+1}.pt")
print("complete")

Epoch 1, Loss: 2396.2289
Epoch 2, Loss: 1861.8971
Epoch 3, Loss: 1422.7875
Epoch 4, Loss: 1055.9580
Epoch 5, Loss: 794.7183
Epoch 6, Loss: 613.8639
Epoch 7, Loss: 486.0420
Epoch 8, Loss: 390.7738
Epoch 9, Loss: 327.9866
Epoch 10, Loss: 280.6229
complete


In [193]:
seq_len = 5

In [194]:
def predict_next_word(seed_text):
    model.eval()
    seed_words = preprocess_text(seed_text)[-seq_len:]
    encoded_input = encode_words(seed_words, vocab)
    if len(encoded_input) < seq_len:
        encoded_input = [0] * (seq_len - len(encoded_input)) + encoded_input
    input_tensor = torch.tensor([encoded_input])
    with torch.no_grad():
        output = model(input_tensor)
        next_word_id = torch.argmax(output, dim=-1).item()
    return idx2word[next_word_id]

In [195]:
print("Input:", "she was not a bit")
print("Next word prediction:", predict_next_word("she was not a bit"))

Input: she was not a bit
Next word prediction: she


In [196]:
print("Input:", "Alice fell down the rabbit")
print("Next word prediction:", predict_next_word("Alice fell down the rabbit"))

Input: Alice fell down the rabbit
Next word prediction: hastily


In [197]:
print("Input:", "I am going to attack")
print("Next word prediction:", predict_next_word("I am going to attack"))

Input: I am going to attack
Next word prediction: me


In [ ]:
from bert_score import score as bertscore
import torch

# Ensure vocab mappings are defined
word2idx = vocab
idx2word = {i: w for w, i in word2idx.items()}

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Replace these with your actual test prefixes
test_inputs = [
    "She walked into the room and",
    "The sun was setting behind",
    "He opened the book and",
    "They ran through the field while",
    "A dog barked at the",
    "He looked at her and",
    "It started to rain just as",
    "The child smiled when",
    "The quick brown fox",
    "He never expected to see"
]

# Replace these with correct ground-truth words if you have them
true_words = [
    "smiled", "mountains", "read", "laughing", "mailman",
    "nodded", "arrived", "joy", "jumps", "her"
]

predicted_sentences = []
true_sentences = []

for input_text, true_word in zip(test_inputs, true_words):
    input_tokens = input_text.strip().split()
    input_ids = torch.tensor([[word2idx.get(w, word2idx["<UNK>"]) for w in input_tokens]]).to(device)

    with torch.no_grad():
        output = model(input_ids)  # expected shape: [1, vocab_size]
        predicted_token_id = output.argmax(dim=-1).item()  # no need for [:, -1, :]
        predicted_word = idx2word.get(predicted_token_id, "<UNK>")

    predicted_sentence = input_text + " " + predicted_word
    #predicted_sentence = input_text + " foobar"
    true_sentence = input_text + " " + true_word

    predicted_sentences.append(predicted_sentence)
    true_sentences.append(true_sentence)

# Compute BERTScore
P, R, F1 = bertscore(predicted_sentences, true_sentences, lang="en", verbose=True)

# Print each result
for i, (inp, pred, true, score) in enumerate(zip(test_inputs, predicted_sentences, true_sentences, F1)):
    print(f"{i+1}. Input: {inp}")
    print(f"   True Sentence:      {true}")
    print(f"   Predicted Sentence: {pred}")
    print(f"   BERTScore (F1):     {score.item():.4f}")
    print("-" * 60)

# Print average BERTScore
average_score = F1.mean().item()
print(f"\n Average BERTScore over 10 examples: {average_score:.4f}")


calculating scores...
computing bert embedding.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.42s/it]


computing greedy matching.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 546.35it/s]

done in 2.42 seconds, 4.13 sentences/sec
1. Input: She walked into the room and
   True Sentence:      She walked into the room and smiled
   Predicted Sentence: She walked into the room and cried
   BERTScore (F1):     0.9713
------------------------------------------------------------
2. Input: The sun was setting behind
   True Sentence:      The sun was setting behind mountains
   Predicted Sentence: The sun was setting behind cried
   BERTScore (F1):     0.9580
------------------------------------------------------------
3. Input: He opened the book and
   True Sentence:      He opened the book and read
   Predicted Sentence: He opened the book and cried
   BERTScore (F1):     0.9572
------------------------------------------------------------
4. Input: They ran through the field while
   True Sentence:      They ran through the field while laughing
   Predicted Sentence: They ran through the field while cried
   BERTScore (F1):     0.9604
-----------------------------------------